In [1]:
import numpy as np
import IPython
import scipy.signal as signal
import matplotlib.pyplot as plt
import librosa
import soundfile as sf
import urllib
from librosa.display import specshow
!apt install ffmpeg
!pip install ffmpeg-python

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [2]:
#@title recording
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io

import ffmpeg
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr
def plot_spec(data, rate, n_fft=512, hop_length=256, win_length=256, yscale='linear'):
    plt.clf()
    spec = librosa.amplitude_to_db(np.abs(librosa.stft(data, n_fft=n_fft, hop_length=hop_length, win_length=win_length)))
    specshow(spec, sr=rate, y_axis=yscale, x_axis='time', cmap='viridis')
    plt.colorbar()
    plt.show()

In [3]:
def plot_spec(data, rate, n_fft=512, hop_length=256, win_length=256, yscale='linear'):
    plt.clf()
    spec = librosa.amplitude_to_db(np.abs(librosa.stft(data, n_fft=n_fft, hop_length=hop_length, win_length=win_length)))
    specshow(spec, sr=rate, y_axis=yscale, x_axis='time', cmap='viridis')
    plt.colorbar()
    plt.show()

In [4]:
# y, sr = get_audio()
# y = y/np.max(np.abs(y))
url = "https://raw.githubusercontent.com/timsainb/noisereduce/master/assets/fish.wav"
response = urllib.request.urlopen(url)
data, sr1 = sf.read(io.BytesIO(response.read()))
y1 = data

In [5]:
url2 = 'https://www2.cs.uic.edu/~i101/SoundFiles/StarWars60.wav'
response = urllib.request.urlopen(url2)
data, sr2 = sf.read(io.BytesIO(response.read()))
y2 = data

In [6]:
IPython.display.Audio(y1, rate=sr1)

In [7]:
IPython.display.Audio(y2, rate=sr2)

Output hidden; open in https://colab.research.google.com to view.

In [8]:
y11 = 0.3*y1 + 0.7*y2[:len(y1)]
y22 = 0.7*y1 + 0.3*y2[:len(y1)]

In [9]:
IPython.display.Audio(y11, rate=sr1)

In [10]:
IPython.display.Audio(y22, rate=sr1)

In [11]:
from sklearn.decomposition import FastICA, PCA

# Compute ICA
ica = FastICA(n_components=2, whiten="arbitrary-variance")
S_ = ica.fit_transform(np.array([y11, y22]).T)  # Reconstruct signals
S_.shape

(200542, 2)

In [12]:
IPython.display.Audio(S_[:, 1], rate=sr1)

In [13]:
IPython.display.Audio(S_[:, 0], rate=sr1)